Импортируем нужные библиотеки.

In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import datetime as dt

In [2]:
!pip install --user --upgrade catboost

Requirement already up-to-date: catboost in c:\programdata\anaconda3\lib\site-packages (0.22)


Распакуйте архив с данными в папку,где находится этот jupyter notebook (baseline.ipynb). У вас будет папка data,  содержащая необходимые файлы. 

Ваша задача предсказать вероятность совершения покупки Клиентом в определенных 8 категориях в следующие 7 дней, чтобы под них Банк смог направить релевантный контент (подборки). Для подготовки предсказания у вас есть данные по транзакциям 50 000 Клиентов в течение года, из которых 25 000 – в обучающей выборке и 25 000 – в тестовой. Ваша задача для тестовой выборки рассчитать для каждой из целевой категории вероятность совершения покупки в ней на следующей неделе.

Данные по транзакциям находятся в файле **transactions_train.csv**. Информация о покупках по категориям для исследуемой недели находится в файле **train_target.csv**.

Считаем данные по транзакциям и правильные ответы.

In [3]:
transactions_train=pd.read_csv('transactions_train.csv')

In [4]:
train_target=pd.read_csv('train_target.csv')

Посмотрим на данные.

In [5]:
transactions_train.head(5)

,client_dk,trans_date,amount,small_group
0,43976,0,4.563,2
1,8417,0,48.342,0
2,17309,0,12.320,0
3,33523,0,29.005,6
4,24228,0,10.266,6


* client_dk - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* amount - сумма транзакции
* small_group - категория покупки

In [6]:
train_target.head(5)

,client_dk,27,32,41,45,67,73,81,88
0,39762,1,0,0,0,0,0,0,0
1,10586,0,0,0,1,0,0,0,0
2,40115,0,1,0,0,0,0,0,0
3,34543,0,0,0,1,0,0,0,0
4,5372,0,0,0,1,0,0,0,0


In [7]:
zero_day = dt.datetime(2009, 1, 19, 0, 0)
def get_month(nday):
  new_date = zero_day + dt.timedelta(days=nday)
  return new_date.month
  
def get_weekday(nday):
  new_date = zero_day + dt.timedelta(days=nday)
  return new_date.weekday()

def get_features(base, ids, kind):
    cost_features = base.groupby('client_dk')['amount'].agg(['sum','mean','std','min','max'])
    count = transactions_train.groupby('client_dk')['trans_date'].count()

    cost_appearance_features = base.groupby('client_dk')['amount'].agg([lambda x: x[x < 10].count(),
                                                                            lambda x: x[x < 50].count(),
                                                                            lambda x: x[x < 75].count(),
                                                                            lambda x: x[x < 100].count(),
                                                                            lambda x: x[x < 500].count(),
                                                                            lambda x: x[x > 500].count(),
                                                                            lambda x: x[x < 1000].count(),
                                                                            lambda x: x[x > 5000].count(),
                                                                            lambda x: x[x > 10000].count()])
    cost_appearance_features.columns = ['trans_less10', 'trans_less50', 'trans_less75', 'trans_less100', 'trans_less500', 'trans_greater500', 'trans_less1000', 'trans_greater5000', 'trans_greater10000']
    for i in cost_appearance_features.columns:
      cost_appearance_features['weighted_'+i] = cost_appearance_features[i] / count
    
    base["trans_month"] = base["trans_date"].apply([lambda x: get_month(x)])
    base["trans_weekday"] = base["trans_date"].apply([lambda x: get_weekday(x)])
    base["trans_week"] = base["trans_date"].apply(lambda x: (x // 7) % 52)

    counter_mean = base.groupby(['client_dk','trans_date'])['amount'].count()
    counter_mean = pd.DataFrame(counter_mean).groupby('client_dk').mean()
    counter_mean.columns = ['mean_count_per_day']
    
    counter_date = base.groupby(['client_dk',"trans_weekday"])['amount'].count()
    counter_date = counter_date.reset_index().pivot(index='client_dk', \
                                                      columns="trans_weekday",values='amount')
    counter_date = counter_date.fillna(0)
    counter_date.columns = ['weekday_count_' + str(i) for i in range(7)] 
    # количество транзакций по дням
    
    amount_day = base.groupby(['client_dk',"trans_weekday"]).sum()['amount']
    amount_day = amount_day.reset_index().pivot(index='client_dk', \
                                                      columns="trans_weekday",values='amount')
    amount_day = amount_day.fillna(0)
    amount_day.columns = ['weekday_spend_' + str(i) for i in range(7)]

    amount_day['weekend_spend'] = amount_day['weekday_spend_5'] + amount_day['weekday_spend_6']
    amount_day['workday_spend'] = cost_features['sum'] - amount_day['weekend_spend']

    # сумма транзакций по дням
    
    counter_month = base.groupby(['client_dk','trans_month'])['amount'].count()
    counter_month = counter_month.reset_index().pivot(index='client_dk', \
                                                      columns='trans_month',values='amount')
    counter_month = counter_month.fillna(0)
    counter_month.columns = ['month_count_' + str(i) for i in range(12)]
    # количество транзакций по месяцам
    
    amount_month = base.groupby(['client_dk','trans_month']).sum()['amount']
    amount_month = amount_month.reset_index().pivot(index='client_dk', \
                                                      columns='trans_month',values='amount')
    amount_month = amount_month.fillna(0)
    amount_month.columns = ['month_spend_' + str(i) for i in range(12)]
    # сумма транзакций по месяцам
        
    counter_week = base.groupby(['client_dk','trans_week'])['amount'].count()
    counter_week = counter_week.reset_index().pivot(index='client_dk', \
                                                      columns='trans_week',values='amount')
    counter_week = counter_week.fillna(0)
    counter_week.columns = ['week_count_' + str(i) for i in range(52)]
    # количество транзакций по неделям
    
    amount_week = base.groupby(['client_dk','trans_week']).sum()['amount']
    amount_week = amount_week.reset_index().pivot(index='client_dk', \
                                                      columns='trans_week',values='amount')
    amount_week = amount_week.fillna(0)
    amount_week.columns = ['week_spend_' + str(i) for i in range(52)]
    # сумма транзакций по неделям
    
    counter_cat = base.groupby(['client_dk','small_group'])['amount'].count()
    counter_cat = counter_cat.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    counter_cat = counter_cat.fillna(0)
    counter_cat.columns = ['cat_count_' + str(i) for i in counter_cat.columns]
    # количество транзакций по категориям
    
    
    amount_cat = base.groupby(['client_dk','small_group']).sum()['amount']
    amount_cat = amount_cat.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount')
    amount_cat = amount_cat.fillna(0)
    amount_cat.columns = ['cat_spend_' + str(i) for i in amount_cat.columns]
    # сумма транзакций по категориям
    
    output = pd.merge(ids, cost_features.reset_index(), on='client_dk')
    output = pd.merge(output, cost_appearance_features.reset_index(), on='client_dk')
    output = pd.merge(output, counter_mean.reset_index(), on='client_dk')
    output = pd.merge(output, counter_cat.reset_index(), on='client_dk')
    output = pd.merge(output, amount_day.reset_index(), on='client_dk')
    output = pd.merge(output, amount_cat.reset_index(), on='client_dk')
    output = pd.merge(output, counter_date.reset_index(), on='client_dk')
    output = pd.merge(output, counter_month.reset_index(), on='client_dk')
    output = pd.merge(output, amount_month.reset_index(), on='client_dk')
    output = pd.merge(output, counter_week.reset_index(), on='client_dk')
    output = pd.merge(output, amount_week.reset_index(), on='client_dk')  

    counter_date['weekend_count'] = counter_date['weekday_count_5'] + counter_date['weekday_count_6']
    counter_date['workday_count'] = count - counter_date['weekend_count']

    for i in counter_cat.columns:
        output["weighted_" + i] = counter_cat[i] / count
    for i in counter_week.columns:
        output["weighted_" + i] = counter_week[i] / count
    for i in counter_month.columns:
        output["weighted_" + i] = counter_month[i] / count
    for i in counter_date.columns:
        output["weighted_" + i] = counter_date[i] / count

    for i in amount_cat.columns:
        output["weighted_" + i] = amount_cat[i] / output["sum"] 
    for i in amount_day.columns:
        output["weighted_" + i] = amount_day[i] / output["sum"] 
    for i in amount_month.columns:
        output["weighted_" + i] = amount_month[i] / output["sum"] 
    for i in amount_week.columns:
        output["weighted_" + i] = amount_week[i] / output["sum"] 
    output = output.drop("client_dk", 1)
    output = output.fillna(0)
    return output

* client_dk - уникальный идентификатор клиента, соответствует полю client_dk из транзакций
* числовые названия колонок - это 8 категорий продуктов. Их названия (числа) соответствуют значениям в колонке **small_group** из данных по транзакциям. Значения в этих колонках бинарные,т.е. 0 - в исследуемую неделю не было совершено покупки в данной категории, 1 - покупка была совершена. Например, клиент с номером 34110 (первая строчка) купил товар из категории 45, а по остальным категориям покупок не совершал.

Далее представлен простой вариант решения задачи. Вы можете решать соревнование используя совершенно другой подход.

Посчитаем по каждому клиенту самые простые аггрегационные признаки.

Посчитаем для каждого клиента количество транзакций по каждой категории.

Теперь подгрузим тестовые данные для того, чтобы сделать предсказание. Проделаем с ними те же самые манипуляции, как и с обучающими данными.

In [8]:
transactions_test=pd.read_csv('transactions_test.csv')
test_id=pd.read_csv('test.csv')

Также загрузим id тестовых клиентов, по которым нужно сделать предсказание.

In [9]:
train = get_features(transactions_train, train_target, 'train')
test = get_features(transactions_test, test_id, 'test')

In [10]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [11]:
X_train=train[common_features]
X_test=test[common_features]

In [14]:
params = {
    'n_estimators': 1000,
    'depth': 6,
    'learning_rate' : 0.027,
    'eval_metric': 'AUC',
    'loss_function': 'MultiClass',
    'verbose':1000
}

В этом бэйзлайне мы будем использовать простой подход - предсказывать покупки в каждой категории независимо. То есть в цикле модель обучается на отдельную категорию как на зависимую переменную, и пытается предсказать наличие покупки в этой определенной категории для теста. В итоге у нас получается 8 задач бинарной классификации.

**Важно**: Такой подход не претендует на звание лучшего, вы вольны придумать свой алгоритм решения, который, вполне вероятно, окажется лучше.

In [15]:
#В словарь будем записывать предсказания модели
results_tree = {}
#Цикл со второго элемента, потому что первой колонкой идет идентификатор клиента 
for q in train_target.columns[1:]:
    print('train product '+str(q))
    curr_target_train = train_target.loc[:,q]
    model = CatBoostClassifier(random_state=42, **params)
    model.fit(X_train.fillna(0).values,curr_target_train.values)
    #Сделаем предсказание
    pred = model.predict_proba(X_test.fillna(0).values)[:,1]
    results_tree[q] = pred

train product 27
0:	total: 251ms	remaining: 4m 11s
999:	total: 3m 56s	remaining: 0us
train product 32
0:	total: 270ms	remaining: 4m 29s
999:	total: 4m	remaining: 0us
train product 41
0:	total: 264ms	remaining: 4m 23s
999:	total: 4m 3s	remaining: 0us
train product 45
0:	total: 275ms	remaining: 4m 34s
999:	total: 4m	remaining: 0us
train product 67
0:	total: 266ms	remaining: 4m 25s
999:	total: 3m 51s	remaining: 0us
train product 73
0:	total: 289ms	remaining: 4m 49s
999:	total: 3m 50s	remaining: 0us
train product 81
0:	total: 255ms	remaining: 4m 14s
999:	total: 3m 52s	remaining: 0us
train product 88
0:	total: 260ms	remaining: 4m 20s
999:	total: 3m 58s	remaining: 0us


Такое решение дает на публичном лидерборде качество 0.6023

### Подготовим файл для отправки в систему

In [16]:
submission = pd.DataFrame(data=np.zeros((25000,8)),columns=train_target.columns[1:],index=test_id['client_dk'].values)

In [17]:
for q in results_tree:
    submission[q] = results_tree[q]
submission.columns = ['cat_27','cat_32','cat_41','cat_45','cat_67','cat_73','cat_81','cat_88']

In [18]:
submission.index.name = 'client_dk'

Сохраняем прогноз на диск в папку submissions. Имя прогноза соответсвует дате и времени его создания, закодированными с помощью timestamp.

In [19]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

submissions/1583401743.csv


Теперь все готово! Можно отправлять решение.